# Check EDR Server Queries

Check that all queries supported by the EDR Server are functioning correctly. This is not really a test suite as (a) it's not automated and (b) it's neither complete nor checks error cases. What it does provide is some form of assurance that all expected queries are supported and running properly, and provides a reminder of how to call the different queries supported by the server.

## Setup

In [ ]:
import json
import random

import requests
from shapely import wkt
from shapely.geometry import MultiPoint, Point, Polygon

In [ ]:
host = "http://localhost:8808"

## Checks

Now run through all the checks.

### Basic connectivity

Checks if the server is running. Should return a capabilities response (metadata about the server / host institute).

In [ ]:
requests.get(host).content

### Collections

Check the get collections request, which should return a JSON response of all the collections served.

In [ ]:
query = f"{host}/collections"
response = requests.get(query).content
print(f"Number of collections: {len(json.loads(response)['collections'])}\n")
print(response)

### Individual Collection

Check we can query an individual collection by retrieving the list of collection IDs from the previous query and requesting one of them at random. This should return a JSON response of a single collection.

**Not working!!**

In [ ]:
collection_ids = [c["id"] for c in json.loads(response)['collections']]
random_collection_id = collection_ids[random.randint(0, len(collection_ids)-1)]
query = f"{host}/collections/{random_collection_id}"
response = requests.get(query).content

print(f"{collection_ids}\n")
print(response)

### Locations

Check we can query an individual collection for its list of locations. This should return a JSON object of type `FeatureCollection` with one or more locations listed in it.

In [ ]:
query = f"{host}/collections/00004/locations"
response = requests.get(query).content

print(f"Number of locations: {len(json.loads(response)['features'])}\n")
print(response)

### Individual Location

Check we can query an individual location in a collection. The response should be a JSON object of type `Domain` containing metadata for an individual location.

In [ ]:
location_ids = [l["id"] for l in json.loads(response)['features']]
random_location_id = location_ids[random.randint(0, len(location_ids)-1)]
query = f"{host}/collections/{random_collection_id}/locations/{random_location_id}"
response = requests.get(query).content

print(f"{location_ids}\n")
print(response)

### Items

Check we can query an individual location for its list of items. This should return a JSON object of type `FeatureCollection` containing one or more individual items.

In [ ]:
query = f"{host}/collections/{random_collection_id}/items"
response = requests.get(query).content

print(response)

### Individual Item

Check we can make a data request to get data values that describe a single item, selected specifically from the dummy dataset for the sake of simplicity. The response should be data-containing (coverage)JSON of type `TiledNdArray`.

In [ ]:
query = f"{host}/collections/00002/items/Parameter 4_0_0"
response = requests.get(query).content

print(response)

### Area

Check we can make a query to return data within a certain area. Note that this can mean either:
* "return the features that intersect with the area", or
* "return the data values for all features that are contained within the area"

The choice on how to implement this is left to the data interface. In the case of the dummy dataset, the first option is chosen. This should return a JSON response of type `Domain` containing one or more features that intersect with the polygon.

In [ ]:
polygon = Polygon([(-0.9, 51.2), (0.3, 51.2), (0.3, 51.7), (-0.9, 51.7), (-0.9, 51.2)])
query_str = f"coords={wkt.dumps(polygon)}"
query = f"{host}/collections/00004/area?{query_str}"
response = requests.get(query).content

print(response)

### Radius

The radius query is identical in intention and response to the area query, other than that the polygon is defined as a circle with a defined radius from a point.

In [ ]:
point = Point(-0.2, 51.5)
within = 25
query_str = f"coords={wkt.dumps(point)}&within={within}&within_units=km"
query = f"{host}/collections/00005/radius?{query_str}"
response = requests.get(query).content

print(response)

### Position

The position query is similar to radius and area, but the coordinates may only be an individual point location, or a list of points (in a `MultiPoint` geometry). This should return a JSON response of type `Domain`. 

In [ ]:
point = Point(0.1, 51.5)
query_str = f"coords={wkt.dumps(point)}"
query = f"{host}/collections/00005/position?{query_str}"
response = requests.get(query).content

print(response)